In [1]:
import pandas as pd
from sklearn.cluster import KMeans
from collections import defaultdict
import warnings

# Suprimir warnings específicos do KMeans sobre nomes de features para simplificar a saída
warnings.filterwarnings("ignore", message="Feature names only support names that are all strings.")
warnings.filterwarnings("ignore", message="X does not have valid feature names, but KMeans was fitted with feature names")


class KMeansMovieRecommender:
    """
    Sistema de recomendação de filmes usando K-Means para agrupar usuários.
    """

    def __init__(self, user_movie_data, k=3):
        """
        Inicializa o sistema de recomendação.

        Args:
            user_movie_data (dict): Dicionário {user_id: set(movie_ids)}.
            k (int): O número de clusters (grupos de usuários) a serem criados.
        """
        if not user_movie_data:
            raise ValueError("Os dados de usuário-filme não podem estar vazios.")

        self.user_movie_data_dict = user_movie_data
        self.k = k
        self.model = None
        self.user_cluster_map = None
        self.user_item_matrix = self._create_user_item_matrix()
        print(f"Sistema inicializado. K={self.k}. Processando {self.user_item_matrix.shape[0]} usuários e {self.user_item_matrix.shape[1]} filmes.")

    def _create_user_item_matrix(self):
        """Cria a matriz usuário-item a partir do dicionário de dados."""
        # Encontra todos os filmes únicos
        all_movies = set()
        for movies in self.user_movie_data_dict.values():
            all_movies.update(movies)
        movie_list = sorted(list(all_movies)) # Garante ordem consistente das colunas

        # Cria o DataFrame: linhas=usuários, colunas=filmes, valores=1 se assistiu, 0 se não
        matrix = pd.DataFrame(0, index=list(self.user_movie_data_dict.keys()), columns=movie_list)

        for user, watched_movies in self.user_movie_data_dict.items():
            matrix.loc[user, list(watched_movies)] = 1

        return matrix

    def fit(self):
        """Treina o modelo K-Means para agrupar os usuários."""
        if self.user_item_matrix.empty:
             print("Matriz usuário-item está vazia. Não é possível treinar.")
             return

        # n_init=10 executa o KMeans 10 vezes com centroides diferentes e escolhe o melhor
        # random_state para resultados reproduzíveis
        self.model = KMeans(n_clusters=self.k, random_state=42, n_init=10)
        print("Treinando o modelo K-Means...")
        cluster_labels = self.model.fit_predict(self.user_item_matrix)

        # Mapeia cada usuário ao seu cluster
        self.user_cluster_map = dict(zip(self.user_item_matrix.index, cluster_labels))
        print("Modelo K-Means treinado com sucesso.")
        self._print_cluster_assignments() # Opcional: mostrar quem está em cada cluster

    def _print_cluster_assignments(self):
        """Mostra quais usuários pertencem a cada cluster."""
        if self.user_cluster_map is None:
            print("Modelo ainda não treinado.")
            return

        clusters = defaultdict(list)
        for user, cluster_id in self.user_cluster_map.items():
            clusters[cluster_id].append(user)

        print("\nDistribuição dos Usuários nos Clusters:")
        for cluster_id, users in sorted(clusters.items()):
            print(f"  Cluster {cluster_id}: {', '.join(users)}")
        print("-" * 30)


    def recommend_movies(self, target_user_id, n_recommendations=5):
        """
        Recomenda filmes para um usuário alvo baseado no seu cluster.

        Args:
            target_user_id (str): O ID do usuário para o qual gerar recomendações.
            n_recommendations (int): O número de recomendações a serem geradas.

        Returns:
            list: Uma lista de IDs de filmes recomendados.
        """
        if self.model is None or self.user_cluster_map is None:
            print("Erro: O modelo K-Means precisa ser treinado primeiro. Chame o método fit().")
            return []

        if target_user_id not in self.user_cluster_map:
            print(f"Erro: Usuário '{target_user_id}' não encontrado nos dados ou no modelo treinado.")
            # Alternativa: poderia tentar atribuir ao cluster mais próximo se o usuário for novo
            # self.model.predict(user_vector) -> mas precisa do vetor do novo usuário
            return []

        target_cluster_id = self.user_cluster_map[target_user_id]
        print(f"\nUsuário '{target_user_id}' pertence ao Cluster {target_cluster_id}.")

        # Encontra outros usuários no mesmo cluster
        peer_users = [user for user, cluster in self.user_cluster_map.items()
                      if cluster == target_cluster_id and user != target_user_id]

        if not peer_users:
            print(f"Nenhum outro usuário encontrado no Cluster {target_cluster_id} para gerar recomendações.")
            return []

        print(f"Usuários no mesmo cluster (peers): {', '.join(peer_users)}")

        # Agrega filmes assistidos pelos 'peers'
        recommendation_scores = defaultdict(int) # Contagem de quantas vezes um filme foi visto no cluster
        target_watched_movies = self.user_movie_data_dict.get(target_user_id, set())

        for peer_user in peer_users:
            peer_watched_movies = self.user_movie_data_dict.get(peer_user, set())
            for movie in peer_watched_movies:
                # Considera apenas filmes que o usuário alvo NÃO assistiu
                if movie not in target_watched_movies:
                    recommendation_scores[movie] += 1 # Incrementa a popularidade do filme no cluster

        # Ordena os filmes pela popularidade (contagem) dentro do cluster
        sorted_recommendations = sorted(recommendation_scores.items(),
                                        key=lambda item: item[1],  # Ordena pela contagem (popularidade)
                                        reverse=True)

        # Retorna os N filmes mais populares no cluster que o usuário não viu
        recommended_movies = [movie for movie, score in sorted_recommendations[:n_recommendations]]

        return recommended_movies

# --- Exemplo de Uso ---

# 1. Modelo (Dados de exemplo: os mesmos do exemplo anterior)
user_movie_data = {
    'Alice': {'Matrix', 'Interestelar', 'Blade Runner 2049', 'O Senhor dos Anéis'},
    'Bob': {'Matrix', 'Interestelar', 'Vingadores Ultimato', 'Star Wars V'},
    'Charlie': {'O Poderoso Chefão', 'Clube da Luta', 'Forrest Gump'},
    'David': {'Matrix', 'Blade Runner 2049', 'Interestelar', 'A Origem'},
    'Eve': {'Vingadores Ultimato', 'Star Wars V', 'Guardiões da Galáxia', 'Matrix'},
    'Frank': {'O Senhor dos Anéis', 'Harry Potter 1', 'O Hobbit'},
    'Grace': {'Interestelar', 'A Origem', 'Gravidade', 'Perdido em Marte'}
}

# 2. Criação da instância do sistema e treinamento
#    Vamos tentar K=3 clusters. A escolha de K pode ser otimizada (ex: método do cotovelo)
k_value = 3
recommender_kmeans = KMeansMovieRecommender(user_movie_data, k=k_value)
recommender_kmeans.fit() # Treina o modelo K-Means

# 3. Geração de Recomendações
target_user = 'Alice'
print(f"\nFilmes já assistidos por {target_user}: {recommender_kmeans.user_movie_data_dict.get(target_user, set())}")

recommendations = recommender_kmeans.recommend_movies(target_user, n_recommendations=3)

print(f"\nRecomendações para {target_user} (baseado no Cluster):")
if recommendations:
    for i, movie in enumerate(recommendations):
        print(f"{i+1}. {movie}")
else:
    print("Nenhuma recomendação encontrada.")

# Exemplo para outro usuário
target_user_2 = 'Charlie'
print(f"\nFilmes já assistidos por {target_user_2}: {recommender_kmeans.user_movie_data_dict.get(target_user_2, set())}")
recommendations_2 = recommender_kmeans.recommend_movies(target_user_2, n_recommendations=3)
print(f"\nRecomendações para {target_user_2} (baseado no Cluster):")
if recommendations_2:
    for i, movie in enumerate(recommendations_2):
        print(f"{i+1}. {movie}")
else:
    print("Nenhuma recomendação encontrada.")

Sistema inicializado. K=3. Processando 7 usuários e 15 filmes.
Treinando o modelo K-Means...
Modelo K-Means treinado com sucesso.

Distribuição dos Usuários nos Clusters:
  Cluster 0: Bob, Eve
  Cluster 1: Charlie, Frank
  Cluster 2: Alice, David, Grace
------------------------------

Filmes já assistidos por Alice: {'Interestelar', 'Matrix', 'O Senhor dos Anéis', 'Blade Runner 2049'}

Usuário 'Alice' pertence ao Cluster 2.
Usuários no mesmo cluster (peers): David, Grace

Recomendações para Alice (baseado no Cluster):
1. A Origem
2. Gravidade
3. Perdido em Marte

Filmes já assistidos por Charlie: {'Forrest Gump', 'O Poderoso Chefão', 'Clube da Luta'}

Usuário 'Charlie' pertence ao Cluster 1.
Usuários no mesmo cluster (peers): Frank

Recomendações para Charlie (baseado no Cluster):
1. Harry Potter 1
2. O Senhor dos Anéis
3. O Hobbit
